
<h2> Задание № 1 (4 балла) </h2>
Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier, RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 8 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [46]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier


In [3]:
#добавим лемматизацию 
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [4]:
data = pd.read_csv('avito_category_classification.csv')

In [4]:
data

,category_name,description
0,Автомобили,"отличное состояние,обслужиание в салоне"
1,Детская одежда и обувь,В отличном состоянии. Фирма KIKO. Очень теплый...
2,Предложение услуг,"Изготовление ограждений, перил,качелей, турник..."
3,Автомобили,Автомобиль в отличном техническом состоянии. О...
4,Бытовая техника,"Продается газовая плита ""Гефест"" (Белоруссия) ..."
...,...,...
9893,Товары для детей и игрушки,Чтобы посмотреть весь ассортимент нашего магаз...
9894,Детская одежда и обувь,"Весна,осень.74-80.вопросы можно в вайбер,двухс..."
9895,"Одежда, обувь, аксессуары","Кимоно Green Hill. Состояние отличное, рост ..."
9896,Детская одежда и обувь,Б/у кроссовки на девочку. Носили только в спор...


In [5]:
data['description_norm'] = data['description'].apply(normalize)

In [6]:
data

,category_name,description,description_norm
0,Автомобили,"отличное состояние,обслужиание в салоне",отличный состояние обслужиание в салон
1,Детская одежда и обувь,В отличном состоянии. Фирма KIKO. Очень теплый...,в отличный состояние фирма kiko очень тёплый у...
2,Предложение услуг,"Изготовление ограждений, перил,качелей, турник...",изготовление ограждение перила качели турников...
3,Автомобили,Автомобиль в отличном техническом состоянии. О...,автомобиль в отличный технический состояние од...
4,Бытовая техника,"Продается газовая плита ""Гефест"" (Белоруссия) ...",продаваться газовый плита гефест белоруссия б ...
...,...,...,...
9893,Товары для детей и игрушки,Чтобы посмотреть весь ассортимент нашего магаз...,чтобы посмотреть весь ассортимент наш магазин ...
9894,Детская одежда и обувь,"Весна,осень.74-80.вопросы можно в вайбер,двухс...",весна осень 74-80 вопрос можно в вайбер двухст...
9895,"Одежда, обувь, аксессуары","Кимоно Green Hill. Состояние отличное, рост ...",кимоно green hill состояние отличный рост 160-...
9896,Детская одежда и обувь,Б/у кроссовки на девочку. Носили только в спор...,б у кроссовок на девочка носить только в спортзал


In [8]:
cv = CountVectorizer(tokenizer =lambda x:x.split(), min_df=5, max_df=0.1)
svd = TruncatedSVD(500) #paramater K 

In [9]:
X_sparse = cv.fit_transform(data['description_norm'])
X = svd.fit_transform(X_sparse)

In [10]:
X_sparse.shape

(9898, 4468)

In [11]:
X.shape

(9898, 500)

In [12]:
sum(svd.explained_variance_ratio_)


0.7900773837354403

In [30]:
pipeline_nmf_rf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(40)),
    ('clf', RandomForestClassifier(n_estimators=200, max_depth=6))
])
pipeline_svd_rf = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x:x.split(), ngram_range=(1,2), min_df=5, max_df= 0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])


In [28]:
metrics_nmf_rf, errors_nmf_rf = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_rf)

In [31]:
metrics_svd_rf, errors_svd_rf = eval_table(data['description_norm'], data['category_name'], pipeline_svd_rf)

In [40]:
pipeline_nmf_sgd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(40)),
    ('clf', SGDClassifier(max_iter=500))
])
pipeline_svd_sgd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x:x.split(), ngram_range=(1,2), min_df=5, max_df= 0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier(max_iter=500))
])


In [36]:
metrics_nmf_sgd, errors_nmf_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_sgd)

In [41]:
metrics_svd_sgd, errors_svd_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_svd_sgd)

In [44]:
pipeline_nmf_k = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(40)),
    ('clf', KNeighborsClassifier(n_neighbors=1))
])
pipeline_svd_k = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x:x.split(), ngram_range=(1,2), min_df=5, max_df= 0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier(n_neighbors=1))
])




In [45]:
metrics_nmf_k, errors_nmf_k = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_k)

In [48]:
metrics_svd_k, errors_svd_k = eval_table(data['description_norm'], data['category_name'], pipeline_svd_k)

In [50]:
pipeline_nmf_e = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(40)),
    ('clf', ExtraTreesClassifier(max_depth=3))
])
pipeline_svd_e = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x:x.split(), ngram_range=(1,2), min_df=5, max_df= 0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier(max_depth =3))
])



In [51]:
metrics_nmf_e, errors_nmf_e = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_e)

In [60]:
metrics_svd_e, errors_svd_e = eval_table(data['description_norm'], data['category_name'], pipeline_svd_e)

In [14]:
def eval_table(X, y, pipeline, N=6):
    # зафиксируем порядок классов
    labels = list(set(y))
    
    # метрики отдельных фолдов будет хранить в табличке
    fold_metrics = pd.DataFrame(index=labels)
    # дополнительно также соберем таблицу ошибок
    errors = np.zeros((len(labels), len(labels)))
    
    # создаем стратегию кросс-валидации
    # shuffle=True (перемешивание) - часто критично важно указать
    # т.к. данные могут быть упорядочены и модель на этом обучится
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        # fit-predict как и раньше, но сразу пайплайном
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        # записываем метрику и индекс фолда
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    # таблица для усредненных значений
    # тут мы берем колонки со значениями и усредняем их
    # часто также все метрики сразу суммируют и в конце просто делят на количество фолдов
    # но мы тут помимо среднего также хотим посмотреть на стандартное отклонение
    # чтобы понять как сильно варьируются оценки моделей
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    # добавим одну колонку со средним по всем классам
    result.loc['mean'] = result.mean().round(2)
    # проценты ошибок просто усредняем
    errors /= N
    
    return result, errors

In [52]:
metrics_nmf_rf

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.83,0.09,0.74,0.04,0.78,0.05
Товары для детей и игрушки,0.74,0.07,0.43,0.09,0.54,0.08
"Одежда, обувь, аксессуары",0.44,0.02,0.79,0.03,0.56,0.02
Квартиры,0.93,0.02,0.94,0.01,0.94,0.01
Бытовая техника,0.33,0.52,0.01,0.02,0.02,0.04
Детская одежда и обувь,0.62,0.03,0.67,0.04,0.64,0.03
Телефоны,0.66,0.05,0.59,0.09,0.62,0.05
Ремонт и строительство,0.45,0.27,0.05,0.04,0.08,0.06
Мебель и интерьер,0.29,0.46,0.01,0.01,0.02,0.03
Предложение услуг,0.68,0.18,0.33,0.26,0.38,0.28


In [53]:
metrics_svd_rf

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.86,0.02,0.59,0.05,0.70,0.03
Товары для детей и игрушки,0.83,0.11,0.20,0.03,0.32,0.04
"Одежда, обувь, аксессуары",0.46,0.02,0.76,0.02,0.57,0.02
Квартиры,0.87,0.05,0.89,0.03,0.88,0.03
Бытовая техника,0.82,0.21,0.06,0.03,0.11,0.05
Детская одежда и обувь,0.45,0.02,0.68,0.02,0.55,0.02
Телефоны,0.94,0.04,0.35,0.05,0.51,0.06
Ремонт и строительство,0.78,0.04,0.08,0.02,0.14,0.03
Мебель и интерьер,0.79,0.40,0.04,0.02,0.07,0.04
Предложение услуг,0.78,0.06,0.46,0.06,0.57,0.04


In [54]:
metrics_nmf_sgd

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.71,0.11,0.74,0.29,0.69,0.22
Товары для детей и игрушки,0.59,0.12,0.42,0.17,0.46,0.13
"Одежда, обувь, аксессуары",0.57,0.11,0.74,0.08,0.63,0.06
Квартиры,0.72,0.17,0.98,0.02,0.82,0.12
Бытовая техника,0.13,0.21,0.01,0.02,0.02,0.03
Детская одежда и обувь,0.60,0.08,0.65,0.08,0.61,0.04
Телефоны,0.64,0.10,0.54,0.15,0.57,0.08
Ремонт и строительство,0.22,0.29,0.01,0.02,0.02,0.03
Мебель и интерьер,0.36,0.30,0.11,0.14,0.12,0.12
Предложение услуг,0.64,0.21,0.40,0.29,0.39,0.19


In [55]:
metrics_svd_sgd

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.86,0.09,0.89,0.04,0.87,0.04
Товары для детей и игрушки,0.73,0.08,0.66,0.04,0.69,0.04
"Одежда, обувь, аксессуары",0.70,0.06,0.80,0.05,0.74,0.01
Квартиры,0.96,0.02,0.96,0.02,0.96,0.01
Бытовая техника,0.66,0.06,0.48,0.08,0.55,0.05
Детская одежда и обувь,0.77,0.04,0.74,0.06,0.75,0.02
Телефоны,0.82,0.07,0.79,0.04,0.80,0.02
Ремонт и строительство,0.57,0.04,0.51,0.11,0.53,0.04
Мебель и интерьер,0.71,0.06,0.60,0.05,0.65,0.02
Предложение услуг,0.77,0.03,0.76,0.04,0.76,0.03


In [56]:
metrics_nmf_k

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.75,0.06,0.71,0.09,0.73,0.06
Товары для детей и игрушки,0.38,0.03,0.42,0.05,0.40,0.04
"Одежда, обувь, аксессуары",0.53,0.03,0.51,0.01,0.52,0.01
Квартиры,0.91,0.02,0.89,0.04,0.90,0.03
Бытовая техника,0.18,0.04,0.21,0.05,0.19,0.04
Детская одежда и обувь,0.55,0.02,0.51,0.04,0.53,0.03
Телефоны,0.53,0.04,0.51,0.05,0.52,0.04
Ремонт и строительство,0.23,0.03,0.28,0.05,0.25,0.04
Мебель и интерьер,0.23,0.06,0.24,0.06,0.23,0.06
Предложение услуг,0.52,0.05,0.59,0.04,0.55,0.04


In [57]:
metrics_svd_k

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.64,0.05,0.47,0.05,0.54,0.04
Товары для детей и игрушки,0.33,0.02,0.35,0.05,0.33,0.03
"Одежда, обувь, аксессуары",0.48,0.03,0.54,0.02,0.51,0.02
Квартиры,0.92,0.04,0.70,0.04,0.80,0.03
Бытовая техника,0.27,0.06,0.29,0.09,0.28,0.07
Детская одежда и обувь,0.51,0.01,0.52,0.01,0.52,0.01
Телефоны,0.46,0.03,0.39,0.06,0.42,0.04
Ремонт и строительство,0.24,0.04,0.25,0.06,0.25,0.04
Мебель и интерьер,0.31,0.05,0.31,0.07,0.30,0.06
Предложение услуг,0.54,0.06,0.56,0.05,0.55,0.05


In [58]:
metrics_nmf_e

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.83,0.41,0.16,0.11,0.27,0.17
Товары для детей и игрушки,0.83,0.41,0.10,0.07,0.18,0.11
"Одежда, обувь, аксессуары",0.31,0.02,0.85,0.04,0.46,0.03
Квартиры,1.00,0.00,0.58,0.24,0.71,0.21
Бытовая техника,0.00,0.00,0.00,0.00,0.00,0.00
Детская одежда и обувь,0.55,0.04,0.59,0.06,0.57,0.04
Телефоны,0.00,0.00,0.00,0.00,0.00,0.00
Ремонт и строительство,0.00,0.00,0.00,0.00,0.00,0.00
Мебель и интерьер,0.00,0.00,0.00,0.00,0.00,0.00
Предложение услуг,0.00,0.00,0.00,0.00,0.00,0.00


In [62]:
metrics_svd_e

,precision,precision_std,recall,recall_std,f1,f1_std
Автомобили,0.00,0.00,0.00,0.00,0.00,0.00
Товары для детей и игрушки,0.00,0.00,0.00,0.00,0.00,0.00
"Одежда, обувь, аксессуары",0.32,0.01,0.89,0.02,0.47,0.02
Квартиры,0.45,0.33,0.04,0.04,0.06,0.07
Бытовая техника,0.00,0.00,0.00,0.00,0.00,0.00
Детская одежда и обувь,0.35,0.04,0.43,0.05,0.38,0.04
Телефоны,0.00,0.00,0.00,0.00,0.00,0.00
Ремонт и строительство,0.00,0.00,0.00,0.00,0.00,0.00
Мебель и интерьер,0.00,0.00,0.00,0.00,0.00,0.00
Предложение услуг,0.00,0.00,0.00,0.00,0.00,0.00


вывод: самый лучший модель svd_sgd